<a href="https://colab.research.google.com/github/Kushagra2000/Neural_machine_translation/blob/main/Machine_translation.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [2]:
#imports
%tensorflow_version 1.x
import tensorflow as tf
print(tf.__version__)
from tensorflow import keras
import pandas as pd
import numpy as np
import re
import unicodedata
from sklearn.model_selection import train_test_split
from tensorflow.keras.layers import LSTM, Dense, TimeDistributed, Activation,InputLayer,Embedding
from tensorflow.keras.losses import sparse_categorical_crossentropy,categorical_crossentropy
from tensorflow.keras.models import Model
from tensorflow.keras.optimizers import Adam
# !pip install -q -U keras-tuner
# import kerastuner as kt 
import IPython
from keras.utils import to_categorical
!pip install bleu
from bleu import list_bleu

tf.test.gpu_device_name()

1.15.2


'/device:GPU:0'

In [3]:
#reading lines from the txt file
num_examples=20000
with open('drive/MyDrive/deu.txt','r',encoding='utf-8') as f:
  lines=(f.readlines())
lines=lines[:num_examples]

In [4]:
#Hyperparameters go here

test_size=0.3
m1_lr=0.001
m2_lr=0.005
m1_lstm_units=64
m2_lstm_units=64
m1_epochs=25
m2_epochs=30
m1_batch_size=128
m2_batch_size=128
m2_embedding_col=64

In [5]:
#reading the dataset
df=pd.read_csv('drive/MyDrive/dataset.csv',index_col=0)

In [6]:
df.head(10)

,eng,ger
0,<SOS> how arrogant ! <EOS>,<SOS> wie arrogant ! <EOS>
1,<SOS> please get tom . <EOS>,<SOS> bitte hol tom . <EOS>
2,<SOS> go look for it . <EOS>,<SOS> geh es suchen ! <EOS>
3,<SOS> she disappeared . <EOS>,<SOS> sie verschwand . <EOS>
4,<SOS> that book is old . <EOS>,<SOS> das buch ist alt . <EOS>
5,<SOS> hold on a moment . <EOS>,<SOS> augenblick mal ! <EOS>
6,<SOS> what is truth ? <EOS>,<SOS> was ist wahrheit ? <EOS>
7,<SOS> ive been better . <EOS>,<SOS> es ging mir schon besser . <EOS>
8,<SOS> why is he here ? <EOS>,<SOS> warum ist er hier ? <EOS>
9,<SOS> hes your friend . <EOS>,<SOS> er ist dein freund . <EOS>


In [7]:
df.tail(10)

,eng,ger
14840,<SOS> here is your bag . <EOS>,<SOS> deine tasche ist hier . <EOS>
14841,<SOS> read it aloud . <EOS>,<SOS> lies es vor . <EOS>
14842,<SOS> i thought hard . <EOS>,<SOS> ich dachte grundlich nach . <EOS>
14843,<SOS> i must obey tom . <EOS>,<SOS> ich muss tom gehorchen . <EOS>
14844,<SOS> we are students . <EOS>,<SOS> wir sind studenten . <EOS>
14845,<SOS> dont leave ! <EOS>,<SOS> gehe nicht weg ! <EOS>
14846,<SOS> he doesnt sleep . <EOS>,<SOS> er schlaft nicht . <EOS>
14847,<SOS> is tom good ? <EOS>,<SOS> ist tom brav ? <EOS>
14848,<SOS> didnt tom smile ? <EOS>,<SOS> tom lachelte nicht ? <EOS>
14849,<SOS> tom felt sad . <EOS>,<SOS> tom fuhlte sich traurig . <EOS>


In [8]:
class Vocab_builder():
  '''
  Builds vocabulary and 
  word to index and index to word dictionaries
  from dataset
  '''
  def __init__(self,lang,series):
    self.lang=lang
    self.data=series
  def tokenize(self,line):
    return line.split(' ')
  def build_vocab(self):
    self.uniq_words=set()
    
    self.maxlen=0
    count=3
    self.num_list=[]
    for index,line in self.data.items():
      self.word_list=self.tokenize(line)
      self.maxlen=max(len(self.word_list),self.maxlen)
      for word in self.word_list:
        if(word not in self.uniq_words and word!='<EOS>' and word!='<SOS>'):
          self.uniq_words.add(word)
          self.num_list.append(count)
          count+=1
      
    self.vocab_list=['<PAD>','<SOS>','<EOS>']+sorted(list(self.uniq_words))
    self.num_list=[0,1,2]+self.num_list
    print("Built vocabulary having {} elements".format(len(self.vocab_list)))
    print("Largest sentence length (with tags):{}".format(self.maxlen))
    return dict(zip(self.vocab_list,self.num_list)),dict(zip(self.num_list,self.vocab_list))






In [9]:
#Objects of Vocab_builder class
eng=Vocab_builder('eng',df['eng'])
ger=Vocab_builder('ger',df['ger'])


In [10]:
#English word to index and index to word dictionaries
eng_w2i,eng_i2w=eng.build_vocab()


Built vocabulary having 3655 elements
Largest sentence length (with tags):9


In [11]:
#checking for special characters in English dictionary
for i in sorted (eng_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . <EOS> <PAD> <SOS> ? a aah abandon abandoned abated abhor able aboard about abroad absent absurd 

In [12]:
#German word to index and index to word dictionaries
ger_w2i,ger_i2w=ger.build_vocab()

Built vocabulary having 4892 elements
Largest sentence length (with tags):13


In [13]:
#checking if the index dictionaries are correct
'geh'==ger_i2w[ger_w2i['geh']]

True

In [14]:
'go'==eng_i2w[eng_w2i['go']]

True

In [15]:
#checking for special characters in German dictionary
for i in sorted (ger_w2i.keys())[:20] :  
     print(i, end = " ") 

! $ - . <EOS> <PAD> <SOS> ? ab abbiegen abend abendbrot abendessen abenteuer aber abgefahren abgekommen abgelaufen abgelehnt abgelenkt 

In [16]:
def sent_to_ind(sentence,lang):
  '''
  Tokenizes a string and
  converts it to an np array of 
  indices and pads the 
  array according to max sentence length
  '''
  ind_list=[]
  if lang=='eng':
    tokens=eng.tokenize(sentence)
    for token in tokens:
      ind_list.append(eng_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
  else:
    tokens=ger.tokenize(sentence)
    for token in tokens:
      ind_list.append(ger_w2i[token])
    while len(ind_list)<max(ger.maxlen,eng.maxlen):
      ind_list.append(0)
    
  return np.array(ind_list)



In [17]:
#Checking correctness of sentence to index conversion
test_lis=sent_to_ind('<SOS> tom wird bald hier sein . was heißt bald ? <EOS>','ger')

print(len(test_lis))
print(test_lis)

13
[   1 4003 4686  309 1988 3604    6 4568 1946  309    7    2    0]


In [18]:
test_lis=sent_to_ind("<SOS> tom will be here soon . how soon ? <EOS>",'eng')
print(len(test_lis))
print(test_lis)

13
[   1 3269 3557  262 1477 2927    6 1551 2927    7    2    0    0]


In [19]:
#splitting the data into training,testing and validation sets

train_x,test_x,train_y,test_y=train_test_split(df['eng'],df['ger'],test_size=0.1,random_state=42)
train_x,val_x,train_y,val_y=train_test_split(train_x,train_y,test_size=0.23,random_state=42)


In [20]:
def sent_to_np(series,lang,translate_mode):
  '''
  Converts a dataframe column to 
  a unsqueezed np array of indexes
  with padding for feeding into NN
  '''
  ret_list=[]
  if translate_mode==False :
    if lang=='eng':
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'eng'))
    else:
      for index,val in series.items():
        ret_list.append(sent_to_ind(val,'ger'))
    
    ret_list=np.array(ret_list)
    return np.expand_dims(ret_list,axis=2)
  else:
    ans=sent_to_ind(series,'eng')
    ans=np.expand_dims(ans,axis=0)
    ans=np.expand_dims(ans,axis=2)
    return ans


In [21]:
train_x=sent_to_np(train_x,'eng',False)
train_y=sent_to_np(train_y,'ger',False)
test_x=sent_to_np(test_x,'eng',False)
test_y=sent_to_np(test_y,'ger',False)
val_x=sent_to_np(val_x,'eng',False)
val_y=sent_to_np(val_y,'ger',False)


In [22]:
#one hot encoding 
train_y=to_categorical(train_y,num_classes=len(ger.vocab_list))
test_y=to_categorical(test_y,num_classes=len(ger.vocab_list))
val_y=to_categorical(val_y,num_classes=len(ger.vocab_list))
print(train_y.shape)
print(test_y.shape)
print(val_y.shape)


(10291, 13, 4892)
(1485, 13, 4892)
(3074, 13, 4892)


In [23]:
# [batch,timesteps,feature]
print(train_x.shape)
print(train_y.shape)

(10291, 13, 1)
(10291, 13, 4892)


In [24]:
print(test_x.shape)
print(test_y.shape)

(1485, 13, 1)
(1485, 13, 4892)


In [25]:
print(val_x.shape)
print(val_y.shape)

(3074, 13, 1)
(3074, 13, 4892)


In [26]:
def translate(sentence,mod,embedded):
  '''
  Function for translating given English sentence
  to German using model predictions
  '''
  ans=""
  preproc_sent=process_text(sentence,True)
  
  model_inp=sent_to_np(preproc_sent,'eng',True)
  if (embedded):
    model_inp=np.squeeze(model_inp,axis=2)
  
  pred=mod.predict(model_inp)
  for i in pred[0]:
    ind=np.argmax(i)
    if(ger_i2w[ind]=='<SOS>' or ger_i2w[ind]=='<EOS>' or ger_i2w[ind]=='<PAD>'):
        continue
    ans+=ger_i2w[ind]
    ans+=" "
  return ans

In [27]:
#training starts here ----------------------------------------------------------

In [28]:

#Removing singleton axis  3rd axis
#for embedding layer
t_x=np.squeeze(train_x,axis=2)
v_x=np.squeeze(val_x,axis=2)
te_x=np.squeeze(test_x,axis=2)
print(t_x.shape)
print(v_x.shape)
print(te_x.shape)

(10291, 13)
(3074, 13)
(1485, 13)


In [29]:

def leaky_relu(x):
    return tf.nn.leaky_relu(x, alpha=0.03)
  
# @tf.function
# def bleu_score(y_true,y_pred):
#   true_ind=[]
#   pred_ind=[]
#   for i in y_pred[0]:
#     ind=np.argmax(i)
#     if(ger_i2w[ind]=='<SOS>' or ger_i2w[ind]=='<EOS>' or ger_i2w[ind]=='<PAD>'):
#         continue
#     pred_ind.append(ger_i2w[ind])
#   for i in y_true[0]:
#     ind=np.argmax(i)
#     if(ger_i2w[ind]=='<SOS>' or ger_i2w[ind]=='<EOS>' or ger_i2w[ind]=='<PAD>'):
#         continue
#     true_ind.append(ger_i2w[ind])
#   return list_bleu([true_ind],pred_ind)


def base_LSTM_model_HP_TUNING(hp):
  '''
  Simple LSTM model for hyperparameter tuning
  '''
  m1_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m1_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05]) 
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model


In [ ]:
def embedding_LSTM_HP_TUNING(hp):
  '''
  LSTM model with embedding layer for hyperparameter tuning
  '''
  m2_lstm_units=hp.Int('units',min_value=32,max_value=128,step=32)
  m2_lr=hp.Choice('learning_rate', values = [0.001,0.003,0.005,0.008,0.01,0.05,0.08])
  embedding_col=hp.Int('output_dim',min_value=32,max_value=160,step=32)
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model
  

In [ ]:
class ClearTrainingOutput(tf.keras.callbacks.Callback):
  def on_train_end(*args, **kwargs):
    IPython.display.clear_output(wait = True)



In [ ]:
#Hyperparameter tuning starts here
#Hyperparameter tuning of base model
tuner = kt.BayesianOptimization(base_LSTM_model_HP_TUNING,
                     objective = 'val_accuracy', 
                     num_initial_points=50,
                     max_trials=15,
                     directory='./',
                     project_name='base_LSTM')


In [ ]:
tuner.search(train_x,train_y, epochs = 25, validation_data = (val_x, val_y), callbacks = [ClearTrainingOutput()])

Trial 16 Complete [00h 02m 53s]
val_accuracy: 0.7552497386932373

Best val_accuracy So Far: 0.765637218952179
Total elapsed time: 00h 47m 45s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps1 = tuner.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps1.get('units')

128

In [ ]:
best_hps1.get('learning_rate')

0.001

In [ ]:
tuner2 = kt.BayesianOptimization(embedding_LSTM_HP_TUNING,
                     objective = 'val_accuracy', 
                     num_initial_points=50,
                     max_trials=30,
                     directory='./',
                     project_name='embedding')


31


In [ ]:
tuner2.search(t_x,train_y, epochs = 15, validation_data = (v_x, val_y), callbacks = [ClearTrainingOutput()])


Trial 30 Complete [00h 07m 08s]
val_accuracy: 0.8056359887123108

Best val_accuracy So Far: 0.8067874908447266
Total elapsed time: 03h 46m 53s
INFO:tensorflow:Oracle triggered exit


In [ ]:
best_hps2 = tuner2.get_best_hyperparameters(num_trials = 1)[0]

In [ ]:
best_hps2.get('units')

128

In [ ]:
best_hps2.get('learning_rate')

0.01

In [ ]:
best_hps2.get('output_dim')

96

In [30]:
#Making base model using best hyperparameters
def base_LSTM_model(m1_lstm_units,m1_lr):
  '''
  Simple LSTM model
  '''
  lstm=LSTM(m1_lstm_units,return_sequences=True,activation='tanh')  #LSTM layer with output being hiddent state at time t
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation='softmax')) #Dense layer acting on hidden output at each step to generate predictions
  model=tf.keras.Sequential()
  model.add(InputLayer(train_x.shape[1:]))
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m1_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model

In [48]:
#Making embedding model using hyperparameters
def embedding_LSTM(m2_lstm_units,m2_lr,embedding_col):
  '''
  LSTM model with embedding layer
  '''
  lstm=LSTM(m2_lstm_units,return_sequences=True,activation='tanh')
  print(t_x.shape[1])
  embed=Embedding(len(ger.vocab_list),embedding_col,input_length=t_x.shape[1])
  layer_at_t=TimeDistributed(Dense(len(ger.vocab_list),activation="softmax"))
  model=tf.keras.Sequential()
  model.add(embed)
  model.add(lstm)
  model.add(layer_at_t)

  model.compile(loss=categorical_crossentropy,optimizer=Adam(m2_lr),metrics=['accuracy','MeanSquaredError',tf.keras.metrics.Precision(),tf.keras.metrics.Recall()])
  return model

In [32]:
base_model=base_LSTM_model(128,0.001)
base_model.summary()

Instructions for updating:
If using Keras pass *_constraint arguments to layers.
Model: "sequential"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
lstm (LSTM)                  (None, 13, 128)           66560     
_________________________________________________________________
time_distributed (TimeDistri (None, 13, 4892)          631068    
Total params: 697,628
Trainable params: 697,628
Non-trainable params: 0
_________________________________________________________________


In [ ]:
#training the base model

base_model.fit(train_x,train_y,m1_batch_size,m1_epochs,validation_data=(val_x,val_y))


Instructions for updating:
Use tf.where in 2.0, which has the same broadcast rule as np.where
Train on 10291 samples, validate on 3074 samples
Epoch 1/25
10291/10291 [==============================] - 6s 630us/sample - loss: 4.3647 - acc: 0.5011 - mean_squared_error: 1.4518e-04 - precision: 0.8667 - recall: 0.2817 - val_loss: 2.7100 - val_acc: 0.5436 - val_mean_squared_error: 1.0363e-04 - val_precision: 0.9229 - val_recall: 0.4866
Epoch 2/25
10291/10291 [==============================] - 4s 417us/sample - loss: 2.6187 - acc: 0.6354 - mean_squared_error: 1.0137e-04 - precision: 0.9243 - recall: 0.4841 - val_loss: 2.5755 - val_acc: 0.6543 - val_mean_squared_error: 1.0003e-04 - val_precision: 0.9230 - val_recall: 0.4868
Epoch 3/25
10291/10291 [==============================] - 4s 416us/sample - loss: 2.4733 - acc: 0.6553 - mean_squared_error: 9.9661e-05 - precision: 0.9265 - recall: 0.4821 - val_loss: 2.4020 - val_acc: 0.6469 - val_mean_squared_error: 9.8912e-05 - val_precision: 0.9236 - 

In [ ]:
#Evaluating LSTM model on unseen data

l,acc,mse,p,r=base_model.evaluate(test_x,test_y)
print("Base model loss for testing set:{}".format(l))
print("Base model MSE for testing set:{}".format(mse))
print("Base model accuracy for testing set:{}".format(acc))
print("Base model precision for testing set:{}".format(p))
print("Base model recall for testing set:{}".format(r))
print("Base model f1_score for testing set:{}".format((2*p*r)/(p+r)))

1485/1485 [==============================] - 1s 400us/sample - loss: 1.7264 - acc: 0.7080 - mean_squared_error: 7.6027e-05 - precision_7: 0.8947 - recall_6: 0.6224
Base model loss for testing set:1.7263565836530743
Base model accuracy for testing set:0.7080031037330627
Base model precision for testing set:0.8947054743766785
Base model recall for testing set:0.6223776340484619
Base model f1_score for testing set:0.73409907650077


In [ ]:
#Sample translation by base LSTM model
translate("work here",base_model,False)

'tom ist ! ! '

In [55]:
embeded_model=embedding_LSTM(96,0.01,128)
embeded_model.summary()

13
Model: "sequential_7"
_________________________________________________________________
Layer (type)                 Output Shape              Param #   
embedding_6 (Embedding)      (None, 13, 96)            469632    
_________________________________________________________________
lstm_7 (LSTM)                (None, 13, 128)           115200    
_________________________________________________________________
time_distributed_7 (TimeDist (None, 13, 4892)          631068    
Total params: 1,215,900
Trainable params: 1,215,900
Non-trainable params: 0
_________________________________________________________________


In [56]:
#training embedding model
embeded_model.fit(t_x,train_y,batch_size=128,epochs=15,validation_data=(v_x,val_y))

Train on 10291 samples, validate on 3074 samples
Epoch 1/15
10291/10291 [==============================] - 6s 540us/sample - loss: 3.0307 - acc: 0.5968 - mean_squared_error: 1.0956e-04 - precision_7: 0.8333 - recall_7: 0.4982 - val_loss: 2.2306 - val_acc: 0.6865 - val_mean_squared_error: 7.8585e-05 - val_precision_7: 0.9610 - val_recall_7: 0.5674
Epoch 2/15
10291/10291 [==============================] - 4s 434us/sample - loss: 2.0342 - acc: 0.7146 - mean_squared_error: 7.2830e-05 - precision_7: 0.9225 - recall_7: 0.6504 - val_loss: 1.7088 - val_acc: 0.7260 - val_mean_squared_error: 6.8857e-05 - val_precision_7: 0.9119 - val_recall_7: 0.6851
Epoch 3/15
10291/10291 [==============================] - 5s 448us/sample - loss: 1.5261 - acc: 0.7528 - mean_squared_error: 6.3205e-05 - precision_7: 0.9270 - recall_7: 0.7043 - val_loss: 1.4857 - val_acc: 0.7531 - val_mean_squared_error: 6.3955e-05 - val_precision_7: 0.9103 - val_recall_7: 0.7078
Epoch 4/15
10291/10291 [===========================

In [57]:
l,acc,mse,p,r=embeded_model.evaluate(te_x,test_y)
print("Embedded model loss for testing set:{}".format(l))
print("Embedded model accuracy for testing set:{}".format(acc))
print("Embedded model MSE for testing set:{}".format(mse))
print("Embedded model precision for testing set:{}".format(p))
print("Embedded model recall for testing set:{}".format(r))
print("Embedded model f1_score for testing set:{}".format((2*p*r)/(p+r)))


1485/1485 [==============================] - 1s 367us/sample - loss: 1.1855 - acc: 0.8212 - mean_squared_error: 5.2995e-05 - precision_7: 0.8933 - recall_7: 0.7934
Embedded model loss for testing set:1.1855315887566769
Embedded model accuracy for testing set:0.8211862444877625
Embedded model MSE for testing set:5.299515760270879e-05
Embedded model precision for testing set:0.8932695388793945
Embedded model recall for testing set:0.7933695912361145
Embedded model f1_score for testing set:0.8403610485671513


In [ ]:
translate("I am so small",embeded_model,True)

'ich bin so klein . '